In [3]:
import pandas as pd
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import gradio as gr

# Load the CSV containing articles' data
df_articles = pd.read_csv('american_revolution_summaries.csv')

# Initialize the Hugging Face question-answering pipeline
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to get answers and analyze sentiment
def get_answer_and_sentiment(question):
    best_answer = None
    best_score = 0
    best_context = None

    for _, row in df_articles.iterrows():
        context = row['Summary']
        
        # Ensure the context is a string and not empty
        if not isinstance(context, str) or context.strip() == "":
            continue
        
        # Use the QA model to get an answer
        try:
            result = qa_pipeline(question=question, context=context)
        except Exception as e:
            print(f"Error processing context: {e}")
            continue
        
        # Compare the score with the best score so far
        if result['score'] > best_score:
            best_score = result['score']
            best_answer = result['answer']
            best_context = context  # Save the context in case it's needed for debugging

    if best_answer:
        sentiment = analyzer.polarity_scores(best_answer)
        return best_answer, {"Sentiment": sentiment, "Confidence": best_score}
    else:
        return "No relevant content available.", {}

# Gradio Interface
interface = gr.Interface(
    fn=get_answer_and_sentiment,
    inputs=gr.Textbox(label="Question", placeholder="Enter your question here..."),
    outputs=[
        gr.Textbox(label="Answer"),
        gr.JSON(label="Sentiment and Confidence")
    ]
)

interface.launch()


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Q: Who was involved in the American Revolution? Who else?
A: Paul Revere and Samuel Adams
